In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

Import Dataset

In [4]:
customers = pd.read_csv('customer.csv', sep =';')
product = pd.read_csv('product.csv', sep =';')
store = pd.read_csv('store.csv', sep =';')
transaction = pd.read_csv('transaction.csv', sep =';')

In [5]:
customers.head()

,CustomerID,Age,Gender,Marital Status,Income
0,1,55,1,Married,"5,12"
1,2,60,1,Married,"6,23"
2,3,32,1,Married,"9,17"
3,4,31,1,Married,"4,87"
4,5,58,1,Married,"3,57"


In [6]:
product.head()

,ProductID,Product Name,Price
0,P1,Choco Bar,8800
1,P2,Ginger Candy,3200
2,P3,Crackers,7500
3,P4,Potato Chip,12000
4,P5,Thai Tea,4200


In [7]:
store.head()

,StoreID,StoreName,GroupStore,Type,Latitude,Longitude
0,1,Prima Tendean,Prima,Modern Trade,"-6,2","106,816666"
1,2,Prima Kelapa Dua,Prima,Modern Trade,"-6,914864","107,608238"
2,3,Prima Kota,Prima,Modern Trade,"-7,797068","110,370529"
3,4,Gita Ginara,Gita,General Trade,"-6,966667","110,416664"
4,5,Bonafid,Gita,General Trade,"-7,250445","112,768845"


In [8]:
transaction.head()

,TransactionID,CustomerID,Date,ProductID,Price,Qty,TotalAmount,StoreID
0,TR11369,328,01/01/2022,P3,7500,4,30000,12
1,TR16356,165,01/01/2022,P9,10000,7,70000,1
2,TR1984,183,01/01/2022,P1,8800,4,35200,4
3,TR35256,160,01/01/2022,P1,8800,7,61600,4
4,TR41231,386,01/01/2022,P9,10000,1,10000,4


In [9]:
print("Customer Shape: ",customers.shape)
print("Product Shape: ",product.shape)
print("Store Shape: ",store.shape)
print("Transaction Shape: ",transaction.shape)

Customer Shape:  (447, 5)
Product Shape:  (10, 3)
Store Shape:  (14, 6)
Transaction Shape:  (5020, 8)


Merge dataset

In [10]:
merged_df = pd.merge(transaction, product,on='ProductID', suffixes = ('_customers', '_product'))
merged_df = pd.merge(merged_df, store, on='StoreID')

final_merged_df = pd.merge(merged_df, customers, on='CustomerID')
final_merged_df= final_merged_df.drop(['Price_customers'], axis = 1)

final_merged_df.sample(5)

,TransactionID,CustomerID,Date,ProductID,Qty,TotalAmount,StoreID,Product Name,Price_product,StoreName,GroupStore,Type,Latitude,Longitude,Age,Gender,Marital Status,Income
3356,TR43444,240,08/01/2022,P4,5,60000,2,Potato Chip,12000,Prima Kelapa Dua,Prima,Modern Trade,"-6,914864","107,608238",22,1,Single,"3,33"
430,TR34255,431,14/06/2022,P8,1,16000,11,Oat,16000,Sinar Harapan,Prestasi,General Trade,"0,533505","101,447403",40,1,Married,"9,51"
3445,TR31932,70,07/01/2022,P9,4,40000,7,Yoghurt,10000,Buana Indah,Buana,General Trade,"3,316694","114,590111",30,1,Married,"4,42"
4708,TR55582,138,29/11/2022,P5,4,16800,2,Thai Tea,4200,Prima Kelapa Dua,Prima,Modern Trade,"-6,914864","107,608238",37,1,Married,"8,96"
4074,TR2304,121,27/04/2022,P5,5,21000,3,Thai Tea,4200,Prima Kota,Prima,Modern Trade,"-7,797068","110,370529",26,0,Single,"2,93"


Rename all column to lowercase and drop irrelevant column in table

In [12]:
for column in final_merged_df.columns:
    final_merged_df.rename(columns ={column: column.lower()}, inplace = True)
final_merged_df.rename(columns = {'marital status': 'marital_status'}, inplace = True)
final_merged_df.rename(columns = {'product name': 'product_name'}, inplace = True)
final_merged_df.drop(['latitude', 'longitude'], axis=1)
final_merged_df.sample(5)

# change data type
df = final_merged_df
df['date'] = pd.to_datetime(df['date'],format='%d/%m/%Y', infer_datetime_format=True)

df.sample(5)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_27352\1329872331.py:11: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['date'] = pd.to_datetime(df['date'],format='%d/%m/%Y', infer_datetime_format=True)


,transactionid,customerid,date,productid,qty,totalamount,storeid,product_name,price_product,storename,groupstore,type,latitude,longitude,age,gender,marital_status,income
3264,TR34922,380,2022-02-06,P8,1,16000,8,Oat,16000,Sinar Harapan,Harapan Baru,General Trade,"5,54829","95,323753",40,0,Married,"5,77"
5003,TR86426,148,2022-08-15,P10,3,45000,6,Cheese Stick,15000,Lingga,Lingga,Modern Trade,"-5,135399","119,42379",45,0,Married,"19,68"
2434,TR18051,27,2022-07-06,P8,2,32000,5,Oat,16000,Bonafid,Gita,General Trade,"-7,250445","112,768845",43,0,Married,"17,42"
982,TR55343,151,2022-08-18,P4,5,60000,10,Potato Chip,12000,Harapan Baru,Harapan Baru,General Trade,"3,597031","98,678513",42,0,Married,"10,38"
872,TR46911,400,2022-12-16,P9,1,10000,12,Yoghurt,10000,Prestasi Utama,Prestasi,General Trade,"-2,990934","104,756554",23,1,Single,"2,79"


Check for null values

In [13]:
df.isna().sum()

transactionid      0
customerid         0
date               0
productid          0
qty                0
totalamount        0
storeid            0
product_name       0
price_product      0
storename          0
groupstore         0
type               0
latitude           0
longitude          0
age                0
gender             0
marital_status    44
income             0
dtype: int64

Fill null values with mode

In [18]:
fill_marital_status = df['marital_status'].mode()
df['marital_status'].fillna(fill_marital_status, inplace = True)

Create dataframe for clustering

In [19]:
cluster_df = df.groupby('customerid').agg({'transactionid' : 'count',
                                          'qty' : 'sum',
                                          'totalamount' : 'sum'})
cluster_df.head()

,transactionid,qty,totalamount
customerid,,,
1,17,60,623300
2,13,57,392300
3,15,56,446200
4,10,46,302500
5,7,27,268600


Check for outlier

In [ ]:
# Calculate Z-score for each data point
numeric_cols = ['transactionid', 'qty', 'totalamount']
z_scores = np.abs((cluster_df[numeric_cols] - cluster_df[numeric_cols].mean()) / cluster_df[numeric_cols].std())

# Define a threshold for outliers
outlier_threshold = 3

# Identify outliers
outliers = cluster_df[z_scores > outlier_threshold]

# Plot the data
plt.figure(figsize=(12, 6))

# Plot for 'qty'
plt.subplot(1, 3, 1)
plt.scatter(cluster_df.index, cluster_df['qty'], label='qty')
plt.scatter(outliers.index, outliers['qty'], color='red', label='Outliers (qty)')
plt.xlabel('Index')
plt.ylabel('qty')
plt.title('qty')

# Plot for 'transactionid'
plt.subplot(1, 3, 2)
plt.scatter(cluster_df.index, cluster_df['transactionid'], label='transactionid')
plt.scatter(outliers.index, outliers['transactionid'], color='blue', label='Outliers (transactionid)')
plt.xlabel('Index')
plt.ylabel('transactionid')
plt.title('transactionid')

# Plot for 'totalamount'
plt.subplot(1, 3, 3)
plt.scatter(cluster_df.index, cluster_df['totalamount'], label='totalamount')
plt.scatter(outliers.index, outliers['totalamount'], color='green', label='Outliers (totalamount)')
plt.xlabel('Index')
plt.ylabel('totalamount')
plt.title('totalamount')

plt.tight_layout()
plt.show()